In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Download portfolio data
tickers = ['VUAG.L', 'VWRP.L', 'JPM', 'CNDX.L', 'RR.L', 'AAPL', 'NVDA', 'CMG']
data = yf.download(
    tickers=tickers,
    start='2024-01-01',
    end='2025-09-05',
    group_by='ticker',
    auto_adjust=False,
    actions=True,
    threads=True
)
data.to_csv('portfolio_data.csv')

[*********************100%***********************]  8 of 8 completed


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

fx_ticker = "GBPUSD=X"  

end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=365)  
fx_data = yf.download(fx_ticker, start=start_date, end=end_date)
fx_data.to_csv('fx_data.csv')

fx_data.tail()

In [6]:
# Load and process portfolio data
df = pd.read_csv('portfolio_data.csv', header=[0,1], index_col=0, parse_dates=True)

# Flatten columns
df.columns = ['{}_{}'.format(ticker, field) for ticker, field in df.columns]
df.columns = [col.replace('^', '') for col in df.columns]
df.to_csv('flat-portfolio_data.csv')

# Rename CNDX.L to CNDX
df.columns = [col.replace('CNDX.L', 'CNDX') for col in df.columns]

# Fix RR.L prices by dividing by 100
rr_columns = [col for col in df.columns if col.startswith('RR.L_')]
for col in rr_columns:
    df[col] = df[col] / 100

# Remove rows with NaN or 0 values in Adj Close columns
adj_close_columns = [col for col in df.columns if col.endswith('_Adj Close')]
df_cleaned = df.dropna()
df_cleaned = df_cleaned[(df_cleaned[adj_close_columns] != 0).all(axis=1)]
df = df_cleaned

df.to_csv('flat-portfolio_data.csv')

In [7]:
# Load FX data
fx_data = pd.read_csv('fx_data.csv', skiprows=3)
fx_data.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
fx_data['Date'] = pd.to_datetime(fx_data['Date'])
fx_data.set_index('Date', inplace=True)
fx_data['USD_to_GBP'] = 1 / fx_data['Close']

# Load flattened portfolio data
portfolio = pd.read_csv('flat-portfolio_data.csv', index_col=0, parse_dates=True)
all_columns = portfolio.columns
stock_symbols = set([col.split('_')[0] for col in all_columns])
gbp_stocks = [symbol for symbol in stock_symbols if '.L' in symbol]
usd_stocks = [symbol for symbol in stock_symbols if '.L' not in symbol]

# Convert USD stocks to GBP
converted_portfolio = portfolio.copy()
date_count = len(converted_portfolio.index)
for i, date in enumerate(converted_portfolio.index):
    closest_date = fx_data.index[fx_data.index <= date][-1] if any(fx_data.index <= date) else fx_data.index[0]
    conversion_rate = fx_data.loc[closest_date, 'USD_to_GBP']
    for stock in usd_stocks:
        stock_columns = [col for col in converted_portfolio.columns if col.startswith(f"{stock}_")]
        for col in stock_columns:
            if pd.api.types.is_numeric_dtype(converted_portfolio[col]):
                converted_portfolio.loc[date, col] = portfolio.loc[date, col] * conversion_rate
converted_portfolio.to_csv('converted_portfolio_data.csv')
print("Conversion completed. Data saved to 'converted_portfolio_data.csv'")

Conversion completed. Data saved to 'converted_portfolio_data.csv'
